<a href="https://colab.research.google.com/github/micida33/python_codes/blob/main/PET2mov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PETデータから動画を作成
1. PETデータの取り込み
2. トリミング
3. 縮尺変更する
4. 動画化頑張る

# データの準備

ワーキングディレクトリに、画像データのあるフォルダをアップする。
- ディレクトリ名：PET
- ファイル名：00001*.jpg, 00002*.jpg

In [ ]:
# インポート

import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ディレクトリ内のファイル名変更

PathFolder = '/content/PET/' #画像のあるフォルダのパス

os.chdir(PathFolder)
path = '*.jpg'
flist = glob.glob(path)

for file in flist:
  os.rename(file, file[:5] + '.jpg')

count = len(flist)

print(count)
print(flist)

395
['00240.jpg', '00082.jpg', '00019.jpg', '00055.jpg', '00368.jpg', '00011.jpg', '00218.jpg', '00216.jpg', '00213.jpg', '00117.jpg', '00087.jpg', '00370.jpg', '00022.jpg', '00260.jpg', '00035.jpg', '00143.jpg', '00376.jpg', '00313.jpg', '00064.jpg', '00293.jpg', '00276.jpg', '00307.jpg', '00300.jpg', '00168.jpg', '00324.jpg', '00234.jpg', '00299.jpg', '00236.jpg', '00094.jpg', '00076.jpg', '00330.jpg', '00074.jpg', '00186.jpg', '00111.jpg', '00162.jpg', '00083.jpg', '00021.jpg', '00258.jpg', '00328.jpg', '00387.jpg', '00098.jpg', '00295.jpg', '00101.jpg', '00049.jpg', '00005.jpg', '00173.jpg', '00304.jpg', '00369.jpg', '00170.jpg', '00051.jpg', '00332.jpg', '00393.jpg', '00336.jpg', '00159.jpg', '00383.jpg', '00009.jpg', '00092.jpg', '00163.jpg', '00075.jpg', '00066.jpg', '00181.jpg', '00086.jpg', '00301.jpg', '00273.jpg', '00352.jpg', '00212.jpg', '00235.jpg', '00331.jpg', '00084.jpg', '00363.jpg', '00001.jpg', '00337.jpg', '00148.jpg', '00003.jpg', '00013.jpg', '00373.jpg', '00390.

In [ ]:
# もとになるファイルを作る。２次元画像を重ねて３次元にする。

# イメージファイルのパス。1枚目と2枚目を重ねておく！
img0 = cv2.imread(PathFolder + '/00001.jpg', cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread(PathFolder + '/00002.jpg', cv2.IMREAD_GRAYSCALE)
sum0 = np.stack([img0, img1])
sumt = sum0

# 後ろ（前）に二次元画像を重ねていく。

for i in range(2, count):
    img = cv2.imread(PathFolder + '/{0:05d}.jpg'.format(i), cv2.IMREAD_GRAYSCALE)
    sumt = np.concatenate([sumt, [img]], axis=0)

print(sumt.shape)

(395, 512, 512)


In [ ]:
# 確認（冠状断と矢状断）

pixels = sumt
check_cor = pixels[:, int(pixels.shape[1]/2), :]
check_sag = pixels[:, :, int(pixels.shape[2]/2)]
plt.imshow(check_cor)#２つとも必要
plt.show()
plt.imshow(check_sag)
plt.show() 
print(check_cor.shape)
print(check_sag.shape)

In [ ]:
# 画像データのトリミング
# 確認画像より、頭尾側：150-250、腹背側：200-400、左右側：100-400と予想

sumt_trim = sumt[150:250, 200:400, 100:400]
print(sumt_trim.shape)

(100, 200, 300)


In [ ]:
# 再確認（冠状断と矢状断）

pixels_trim = sumt_trim
check_cor = pixels_trim[:, int(pixels_trim.shape[1]/2), :]
check_sag = pixels_trim[:, :, int(pixels_trim.shape[2]/2)]
plt.imshow(check_cor)#２つとも必要
plt.show()
plt.imshow(check_sag)
plt.show() 
print(check_cor.shape)
print(check_sag.shape)

In [ ]:
# ボリュームデータから、冠状断動画を作る

# ボリュームデータの指定
pixels_v = sumt_trim

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')

# ！グレースケールの時は、５つめの変数をFalseにする！
video = cv2.VideoWriter('video.mp4', fourcc, 20.0, (pixels_v.shape[2], pixels_v.shape[0]), False)
MAISU_video = pixels_v.shape[1]
for i in range(0, MAISU_video):
    img = pixels_v[:, i, :]
    img = np.asarray(img)
    img = cv2.resize(img, dsize=(pixels_v.shape[2], pixels_v.shape[0]))
    video.write(img)

video.release()

In [ ]:
# ボリュームデータから、縮尺変更（縦に延長）した、冠状断動画を作る

# ボリュームデータの指定
pixels_v = sumt_trim

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')

# ！グレースケールの時は、５つめの変数をFalseにする！
video = cv2.VideoWriter('video.mp4', fourcc, 20.0, (pixels_v.shape[2], pixels_v.shape[0]*2), False)
MAISU_video = pixels_v.shape[1]
for i in range(0, MAISU_video):
    img = pixels_v[:, i, :]
    img = np.asarray(img)
    img = cv2.resize(img, dsize=(pixels_v.shape[2], pixels_v.shape[0]*2), interpolation=cv2.INTER_LANCZOS4)
    video.write(img)

video.release()